# Extra: Model Comparison
## Minh Nguyen

In [386]:
# Data Cleaning and Plotting Libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams

# Data Analysis Libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut, KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss, roc_auc_score

# Sets parameters for the entire notebook for Seaborn/Matplotlib plots
rcParams['figure.figsize'] = 10, 7
rcParams.update({'font.size': 12})
sns.set_style('darkgrid')

In [387]:
df = pd.read_csv('heart_disease.csv')
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,death
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [388]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  death                     299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [389]:
df["death"].value_counts()

0    203
1     96
Name: death, dtype: int64

## Use Logistic Regression model

In [390]:
X=df.drop('death', axis=1)
y=df['death']

mm_scaler = MinMaxScaler()

X_mm = pd.DataFrame(mm_scaler.fit_transform(X), columns=X.columns)

train_x, test_x, train_y, test_y = train_test_split(X_mm, y, test_size=0.2, stratify=y)

train_x.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
134,0.745455,0.0,0.576295,0.0,0.318182,0.0,0.249606,0.076404,0.685714,1.0,1.0,0.366548
224,0.327273,0.0,0.071319,1.0,0.166667,0.0,0.580555,0.056180,0.714286,1.0,0.0,0.715302
236,0.636364,0.0,0.012248,0.0,0.545455,1.0,0.270215,0.067416,1.000000,1.0,0.0,0.729537
267,0.290909,1.0,0.014289,1.0,0.363636,0.0,0.130804,0.134831,0.771429,1.0,0.0,0.854093
122,0.363636,0.0,0.009314,1.0,0.363636,0.0,0.245969,0.028090,0.771429,0.0,0.0,0.323843


In [391]:
lr_model = LogisticRegression(max_iter=1000)

lr_model.fit(train_x, train_y)
pred = lr_model.predict(test_x)

In [392]:
print("Accuracy: ", accuracy_score(test_y, pred))
print("Confusion Matrix: \n", confusion_matrix(test_y, pred))

Accuracy:  0.8333333333333334
Confusion Matrix: 
 [[38  3]
 [ 7 12]]


- As we can see, the model is pretty good.

## Apply LOOCV

In [393]:
loo = LeaveOneOut()
# loo.get_n_splits(train_x)
loo.get_n_splits(X_mm)

299

In [394]:
accuracy = []
neg_log_loss = []
roc_auc = []

for train_index, test_index in loo.split(X_mm):
    # print("train:", train_index, "validation:", test_index)
    X_train, X_test = X_mm.values[train_index], X_mm.values[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # lr_model1 = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train, y_train)
    y_pred = lr_model.predict(X_test)
    # y_pred_prob = lr_model.predict_proba(X_test)
    accuracy.append(accuracy_score(y_test, y_pred))
    # neg_log_loss.append(log_loss(y_test, y_pred_prob))
    # roc_auc.append(roc_auc_score(y_test, y_pred_prob))

# calculate the mean of the metrics
accuracy_mean = np.mean(accuracy)
# neg_log_loss_mean = np.mean(neg_log_loss)
# roc_auc_mean = np.mean(roc_auc)

print(f"Accuracy: {accuracy_mean}")
# print(f"Neg Log Loss: {neg_log_loss_mean}")
# print(f"ROC AUC: {roc_auc_mean}")

Accuracy: 0.8294314381270903


## Apply 10 and 3 K-Fold

### k = 10

In [395]:
# kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None)
kf = KFold(n_splits=10, shuffle=True)

kf_accuracies_10 = []

for train_index, test_index in kf.split(X_mm):
    # print("Train:", train_index, "Validation:",test_index)
    X_train, X_test = X_mm.values[train_index], X_mm.values[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    
    lr_model.fit(X_train, y_train)
    y_pred = lr_model.predict(X_test)
    kf_accuracies_10.append(accuracy_score(y_test, y_pred))
    
mean_accuracy_10 = np.mean(kf_accuracies_10)

print(f"Accuracy: {mean_accuracy_10}")

Accuracy: 0.825977011494253


In [396]:
kf = KFold(n_splits=3, shuffle=True)

kf_accuracies_3 = []

for train_index, test_index in kf.split(X_mm):
    # print("Train:", train_index, "Validation:",test_index)
    X_train, X_test = X_mm.values[train_index], X_mm.values[test_index] 
    y_train, y_test = y[train_index], y[test_index]
    
    lr_model.fit(X_train, y_train)
    y_pred = lr_model.predict(X_test)
    kf_accuracies_3.append(accuracy_score(y_test, y_pred))
    
mean_accuracy_3 = np.mean(kf_accuracies_3)

print(f"Accuracy: {mean_accuracy_3}")

Accuracy: 0.8025252525252524
